In [1]:
import tensorflow as tf
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt

In [2]:
cat_dogs_ds_url = 'https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip'
path_to_zip = tf.keras.utils.get_file('cats_and_dogs.zip',origin=cat_dogs_ds_url, extract=True)
cat_dogs_ds_dir = os.path.join(os.path.dirname(path_to_zip),'cats_and_dogs_filtered')

In [3]:
train_dir = os.path.join(cat_dogs_ds_dir, 'train')
validation_dir = os.path.join(cat_dogs_ds_dir, 'validation')

In [4]:
train_cats_dir = os.path.join(train_dir, 'cats')
train_dogs_dir = os.path.join(train_dir, 'dogs')  
validation_cats_dir = os.path.join(validation_dir, 'cats') 

In [5]:
validation_dogs_dir = os.path.join(validation_dir, 'dogs')

In [6]:
batch_size = 32
epochs = 1
img_height = 224
img_width = 224

In [7]:
train_image_generator = ImageDataGenerator(rescale=1./255)
train_data_gen = train_image_generator.flow_from_directory(batch_size=batch_size,directory=train_dir,shuffle=True,target_size=(img_height, img_width),class_mode='binary')

Found 2000 images belonging to 2 classes.


In [8]:
validation_image_generator = ImageDataGenerator(rescale=1./255)

In [9]:
val_data_gen = validation_image_generator.flow_from_directory(batch_size=batch_size,directory=validation_dir,target_size=(img_height, img_width),class_mode='binary')

Found 1000 images belonging to 2 classes.


In [10]:
untouched_base_model = tf.keras.applications.VGG16(weights = 'imagenet',include_top=True, input_shape=(img_width, img_height, 3))

In [11]:
 base_model = tf.keras.applications.VGG16(weights = 'imagenet',include_top=False, input_shape=(img_width, img_height, 3)) 

In [12]:
 base_model.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [13]:
base_model.trainable = False 

In [14]:
from tensorflow.keras.layers import GlobalAveragePooling2D 
from tensorflow.keras.layers import Dense 
global_average_layer = GlobalAveragePooling2D(input_shape=base_model.output_shape[1:]) 
prediction_layer = Dense(2, activation="softmax") 

In [15]:
 from tensorflow.keras.models import Sequential 

In [16]:
model = Sequential([ base_model, global_average_layer, prediction_layer ]) 

In [17]:
 model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy']) 

In [18]:
model.fit_generator(train_data_gen, epochs=epochs, validation_data = val_data_gen)


63/63 [==============================] - 964s 15s/step - loss: 0.6530 - accuracy: 0.6315 - val_loss: 0.5904 - val_accuracy: 0.7700


In [19]:
model.save('model.h5')